# Traffic Sign Detection - Modern Deep Learning Methods

This notebook implements and compares state-of-the-art deep learning approaches:
1. YOLOv11 (Ultralytics)
2. Faster R-CNN

**Important**: This notebook requires GPU for efficient training. Use Google Colab with GPU runtime.

## 1. Setup Environment

In [3]:
# Check for GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠ WARNING: No GPU detected. Training will be very slow!")

torch.mps.is_available()

PyTorch version: 2.9.1
CUDA available: False
⚠ WARNING: No GPU detected. Training will be very slow!


True

In [4]:
# Setup environment
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    !git clone https://github.com/YOUR_USERNAME/traffic-sign-detection.git
    %cd traffic-sign-detection
else:
    import os
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')

# Install dependencies
!pip install -q ultralytics roboflow tensorboard

In [5]:
# Imports
import sys
sys.path.append('src')

import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import yaml
import time
from tqdm import tqdm

from src.modern.yolo.trainer import YOLOTrainer
from src.modern.faster_rcnn.trainer import FasterRCNNTrainer
from src.utils.visualization import ResultVisualizer

print("Imports successful!")

Imports successful!


## 2. Download and Prepare Dataset

In [8]:
from src.utils.roboflow_loader import RoboflowDataLoader

# Download dataset
loader = RoboflowDataLoader(
    api_key="gbb6oBUEJlstBEF0CruH",
    workspace="giaothong-t5tdy",
    project="phat_hien_bien_bao-zsswb",
    version=1
)

# Download in YOLOv8 format for YOLO training
yolo_path = loader.download_dataset(format="yolov8")
print(f"YOLO dataset: {yolo_path}")

# Get dataset info
dataset_info = loader.get_dataset_info()
class_names = dataset_info['class_names']
num_classes = dataset_info['num_classes']

print(f"\nClasses ({num_classes}): {class_names}")

loading Roboflow workspace...
loading Roboflow project...
Workspace: giaothong-t5tdy
Project: phat_hien_bien_bao-zsswb
Version: 1
Dataset downloaded successfully to: /Users/kietnt/Desktop/Workspace/traffic-sign-detection/data/raw/yolov8
Dataset info saved to: data/raw/dataset_info_yolov8.json
YOLO dataset: /Users/kietnt/Desktop/Workspace/traffic-sign-detection/data/raw/yolov8

Classes (5): ['bien_bao_cam', 'bien_bao_hieu_lenh', 'bien_bao_nguy_hiem_va_canh_bao', 'bien_chi_dan', 'bien_phu']


## 3. YOLOv11 Training

### 3.1 Initialize YOLOv11 Trainer

In [9]:
# Initialize YOLO trainer
# Try different sizes: 'n' (nano), 's' (small), 'm' (medium), 'l' (large)
yolo_trainer = YOLOTrainer(
    model_size='n',  # Start with nano for faster training
    img_size=640,
    device='auto'
)

# Print model info
yolo_trainer.get_model_info()

Initialized YOLOv11N on cpu
Image size: 640
YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


### 3.2 Train YOLOv11

In [10]:
# Training configuration
data_yaml = Path(yolo_path) / 'data.yaml'

# Train the model
yolo_results = yolo_trainer.train(
    data_yaml=str(data_yaml),
    epochs=100,  # Increase for better results
    batch_size=16,  # Adjust based on GPU memory
    patience=20,  # Early stopping
    save_dir='experiments/yolo',
    name='yolov11n_traffic_signs',
    # Additional args
    lr0=0.01,  # Initial learning rate
    lrf=0.01,  # Final learning rate
    warmup_epochs=3,
    augment=True,
)

print("\nTraining completed!")


Starting YOLO Training
Model: YOLOv11N
Epochs: 100
Batch size: 16
Device: cpu
Data config: /Users/kietnt/Desktop/Workspace/traffic-sign-detection/data/raw/yolov8/data.yaml

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.9.1 CPU (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/Users/kietnt/Desktop/Workspace/traffic-sign-detection/data/raw/yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, 

KeyboardInterrupt: 

### 3.3 Validate YOLOv11

In [ ]:
# Validate on test set
val_results = yolo_trainer.validate(
    data_yaml=str(data_yaml),
    split='test'
)

print("\n" + "="*60)
print("YOLOv11 VALIDATION RESULTS")
print("="*60)
print(f"mAP@0.5: {val_results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
print("="*60)

### 3.4 Test YOLOv11 Predictions

In [ ]:
# Predict on test images
test_images_dir = Path(yolo_path) / 'test' / 'images'
test_images = list(test_images_dir.glob('*.*'))[:10]  # Sample 10 images

results = yolo_trainer.predict(
    source=str(test_images_dir),
    conf=0.25,
    iou=0.7,
    save=True,
    project='experiments/yolo',
    name='predictions'
)

print(f"Predictions saved to: experiments/yolo/predictions")

# Visualize some predictions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, result in enumerate(results[:6]):
    # Get image with predictions
    img = result.plot()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(img)
    axes[idx].set_title(f"Test Image {idx+1}", fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 3.5 Export YOLOv11 Model

In [ ]:
# Export to ONNX for deployment
onnx_path = yolo_trainer.export(format='onnx')
print(f"Model exported to: {onnx_path}")

# You can also export to other formats:
# - 'torchscript': PyTorch TorchScript
# - 'tflite': TensorFlow Lite
# - 'coreml': Apple CoreML
# - 'engine': TensorRT

## 4. Faster R-CNN Training (Optional - Advanced)

### 4.1 Prepare COCO Format Dataset

In [ ]:
# Download dataset in COCO format
coco_path = loader.download_dataset(format="coco")
print(f"COCO dataset: {coco_path}")

### 4.2 Initialize and Train Faster R-CNN

In [ ]:
from modern.faster_rcnn.trainer import FasterRCNNTrainer, TrafficSignDataset

# Create datasets
train_dataset = TrafficSignDataset(
    images_dir=str(Path(coco_path) / 'train'),
    annotations_file=str(Path(coco_path) / 'train' / '_annotations.coco.json')
)

val_dataset = TrafficSignDataset(
    images_dir=str(Path(coco_path) / 'valid'),
    annotations_file=str(Path(coco_path) / 'valid' / '_annotations.coco.json')
)

print(f"Train dataset: {len(train_dataset)} images")
print(f"Val dataset: {len(val_dataset)} images")

# Initialize trainer (num_classes + 1 for background)
frcnn_trainer = FasterRCNNTrainer(
    num_classes=num_classes + 1,
    pretrained=True,
    device='auto'
)

# Train (this takes longer than YOLO)
frcnn_history = frcnn_trainer.train(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    epochs=50,  # Can reduce for faster training
    batch_size=4,
    learning_rate=0.005,
    save_dir='experiments/faster_rcnn',
    checkpoint_freq=10
)

print("\nFaster R-CNN training completed!")

### 4.3 Visualize Faster R-CNN Training History

In [ ]:
# Plot training history
ResultVisualizer.plot_training_history(
    history=frcnn_history,
    save_path='experiments/faster_rcnn/training_history.png'
)

## 5. Performance Comparison

In [ ]:
import pandas as pd

# Compile results
results_data = {
    'YOLOv11n': {
        'mAP@0.5': val_results.box.map50,
        'mAP@0.5:0.95': val_results.box.map,
        'Precision': val_results.box.mp,
        'Recall': val_results.box.mr,
    }
}

# Visualize comparison
ResultVisualizer.plot_metrics_comparison(
    results=results_data,
    metrics=['mAP@0.5', 'mAP@0.5:0.95', 'Precision', 'Recall'],
    title='Model Performance Comparison',
    save_path='experiments/model_comparison.png'
)

# Print summary table
df_results = pd.DataFrame(results_data).T
print("\n" + "="*80)
print("MODERN METHODS PERFORMANCE SUMMARY")
print("="*80)
print(df_results.to_string())
print("="*80)

## 6. Speed Benchmarking

In [ ]:
# Benchmark inference speed
from utils.metrics import SpeedMetrics

speed_metrics = SpeedMetrics()

# Test YOLOv11 speed
print("Benchmarking YOLOv11 speed...")
for img_path in tqdm(test_images[:50]):
    start = time.time()
    _ = yolo_trainer.predict(source=str(img_path), verbose=False)
    speed_metrics.update(time.time() - start)

yolo_speed = speed_metrics.get_summary()

print("\n" + "="*60)
print("SPEED BENCHMARK")
print("="*60)
print(f"YOLOv11n:")
print(f"  Average FPS: {yolo_speed['avg_fps']:.1f}")
print(f"  Average inference time: {yolo_speed['avg_inference_time_ms']:.2f} ms")
print("="*60)

## 7. Final Comparison: Traditional vs Modern

In [ ]:
# Load traditional methods results for comparison (if available)
import json

traditional_results = {}
traditional_results_path = Path('experiments/traditional_methods_results.json')

# We'll create comprehensive comparison combining both notebooks
complete_results = {
    'Traditional - HOG+SVM': {
        'Type': 'Traditional',
        'mAP@0.5': 'See Notebook 02',
        'Precision': 'See Notebook 02',
        'Recall': 'See Notebook 02',
        'Speed': 'Medium (CPU)',
        'Deployment': 'Easy (CPU)',
        'Training Time': 'Minutes',
        'GPU Required': 'No'
    },
    'Traditional - Color+Shape': {
        'Type': 'Traditional',
        'mAP@0.5': 'See Notebook 02',
        'Precision': 'See Notebook 02',
        'Recall': 'See Notebook 02',
        'Speed': 'Very Fast (CPU)',
        'Deployment': 'Very Easy (CPU)',
        'Training Time': 'None',
        'GPU Required': 'No'
    },
    'Modern - YOLOv11n': {
        'Type': 'Deep Learning',
        'mAP@0.5': f"{val_results.box.map50:.4f}",
        'Precision': f"{val_results.box.mp:.4f}",
        'Recall': f"{val_results.box.mr:.4f}",
        'Speed': f"{yolo_speed['avg_fps']:.1f} FPS",
        'Deployment': 'Moderate (GPU recommended)',
        'Training Time': 'Hours (GPU)',
        'GPU Required': 'Recommended'
    }
}

df_complete = pd.DataFrame(complete_results).T

print("\n" + "="*100)
print("COMPLETE PERFORMANCE COMPARISON: TRADITIONAL VS MODERN METHODS")
print("="*100)
print(df_complete.to_string())
print("="*100)

# Create visual comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# mAP comparison (only YOLOv11 for now, others from notebook 02)
methods = ['YOLOv11n']
map_values = [val_results.box.map50]
precision_values = [val_results.box.mp]
recall_values = [val_results.box.mr]

x = np.arange(len(methods))
width = 0.25

axes[0].bar(x - width, map_values, width, label='mAP@0.5', alpha=0.8)
axes[0].bar(x, precision_values, width, label='Precision', alpha=0.8)
axes[0].bar(x + width, recall_values, width, label='Recall', alpha=0.8)
axes[0].set_ylabel('Score')
axes[0].set_title('Modern Methods - Detection Metrics')
axes[0].set_xticks(x)
axes[0].set_xticklabels(methods)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim([0, 1.1])

# Add value labels
for i, (m, p, r) in enumerate(zip(map_values, precision_values, recall_values)):
    axes[0].text(i - width, m + 0.02, f'{m:.3f}', ha='center', fontsize=9)
    axes[0].text(i, p + 0.02, f'{p:.3f}', ha='center', fontsize=9)
    axes[0].text(i + width, r + 0.02, f'{r:.3f}', ha='center', fontsize=9)

# Training time comparison (illustrative)
training_info = {
    'HOG+SVM\n(Notebook 02)': {'time': '~2 min', 'gpu': 'No', 'color': 'lightblue'},
    'Color+Shape\n(Notebook 02)': {'time': 'None', 'gpu': 'No', 'color': 'lightgreen'},
    'YOLOv11n': {'time': '~2-4 hours', 'gpu': 'Yes', 'color': 'coral'}
}

methods_train = list(training_info.keys())
colors = [training_info[m]['color'] for m in methods_train]

axes[1].bar(methods_train, [1, 0, 150], alpha=0.6, color=colors)
axes[1].set_ylabel('Training Time (minutes)')
axes[1].set_title('Training Time & Requirements Comparison')
axes[1].grid(axis='y', alpha=0.3)

# Add text annotations
for i, method in enumerate(methods_train):
    gpu_text = f"GPU: {training_info[method]['gpu']}"
    time_text = training_info[method]['time']
    axes[1].text(i, 5, f'{time_text}\n{gpu_text}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('experiments/complete_methods_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Key Insights:")
print("\n1. **Accuracy**: Modern methods (YOLO) achieve significantly higher mAP@0.5 (typically 0.85-0.95)")
print("   - Traditional methods: mAP@0.5 typically 0.20-0.50 (see Notebook 02)")
print("   - YOLOv11: Excellent precision-recall balance")
print("\n2. **Speed**: YOLOv11 offers excellent real-time performance on GPU")
print("   - Traditional Color+Shape: Fastest on CPU but lowest accuracy")
print("   - Traditional HOG+SVM: Slow due to sliding window")
print("   - YOLOv11: Fast on GPU with best accuracy")
print("\n3. **Deployment**: Trade-off between ease and performance")
print("   - Traditional methods: Easy CPU deployment, lower accuracy")
print("   - Modern methods: Need GPU for best performance, higher accuracy")
print("\n4. **Training**: Modern methods require more resources but deliver superior results")
print("   - Traditional: Minutes to hours on CPU")
print("   - Modern: Hours on GPU, but achieves production-grade accuracy")
print("\n✅ **Recommendation**: Use YOLOv11 for production (best accuracy-speed tradeoff)")
print("   - For embedded/CPU-only: Consider quantized YOLO or traditional methods")
print("   - For prototyping: Start with Color+Shape baseline, then move to YOLO")
print("\n📁 Complete comparison plot saved to: experiments/complete_methods_comparison.png")

## 8. Save Best Model and Results

In [ ]:
import json

# Save experiment results
experiment_results = {
    'dataset': {
        'name': 'Traffic Sign Detection',
        'num_classes': num_classes,
        'class_names': class_names,
    },
    'yolov11n': {
        'mAP@0.5': float(val_results.box.map50),
        'mAP@0.5:0.95': float(val_results.box.map),
        'precision': float(val_results.box.mp),
        'recall': float(val_results.box.mr),
        'fps': float(yolo_speed['avg_fps']),
        'inference_time_ms': float(yolo_speed['avg_inference_time_ms']),
    }
}

# Save to JSON
results_path = Path('experiments/experiment_results.json')
results_path.parent.mkdir(parents=True, exist_ok=True)

with open(results_path, 'w') as f:
    json.dump(experiment_results, f, indent=2)

print(f"Results saved to: {results_path}")
print("\n✅ Experiment completed successfully!")
print("\n📁 Check the experiments/ directory for:")
print("   - Trained models")
print("   - Training logs and plots")
print("   - Prediction visualizations")
print("   - Performance metrics")

## Next Steps

1. **Fine-tune hyperparameters**: Experiment with learning rates, augmentation, model sizes
2. **Ensemble methods**: Combine multiple models for better performance
3. **Deploy**: Export to ONNX/TensorRT for production deployment
4. **Real-world testing**: Test on videos or real-time camera feeds
5. **Optimize**: Quantization, pruning for edge deployment